In [1]:
using Pkg; Pkg.update()
#using Pkg; Pkg.resolve()
using SplitApplyCombine
using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random, Plots

using BenchmarkTools

    Updating registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Installed ConstructionBase ──── v1.5.4
   Installed MakieCore ─────────── v0.6.6
   Installed ADTypes ───────────── v0.2.2
   Installed RecursiveArrayTools ─ v2.38.8


   Installed StaticArrays ──────── v1.6.3
   Installed SciMLBase ─────────── v1.97.0


    Updating `~/ParallelTemperingMonteCarlo.jl/Project.toml`
  [90137ffa] ↑ StaticArrays v1.6.2 ⇒ v1.6.3
    Updating `~/ParallelTemperingMonteCarlo.jl/Manifest.toml`


...

  ✓ InteratomicPotentials
  ✓ ReverseDiff


  ✓ ComponentArrays


  ✓ DftFunctionals


  ✓ DFTK


  ✓ ParallelTemperingMonteCarlo
  30 dependencies successfully precompiled in 178 seconds. 108 already precompiled.


# Main Goal

Broadly speaking, each potential requires different variables, vectors and changes at each step, this leads to method double-ups (with nonspecific variable types) as well as the need to rewrite high level functions such as mc_step and mc_cycle. By writing a single vector or struct that contains all the required vectors etc would solve both problems. 

In [2]:
abstract type PotentialVariables end

Each potential requires different vectors, both for sampling and as temporary delta vectors. Starting with the ELJ potential, requiring essentially nothing, maybe the new_dist2_vector if anything


In [7]:
function test_function_individual(x::Float64,y::Float64)
    x += y 
    return x
end
function test_function_vec(xvec,why)
    xvec = test_function_individual.(xvec,why)
    return xvec 
end
x_vec = [1., 2., 3., 4., 5.]
y= [2. , 2. , 3. , 4. , 4. ]

xvec = test_function_vec(x_vec,y)


5-element Vector{Float64}:
 3.0
 4.0
 6.0
 8.0
 9.0

In [3]:
mutable struct DimerPotentialVariables <: PotentialVariables
    new_en::Float64
    new_dist2_vec::Vector
end
function set_variables(config,dist_2_mat,pot::AbstractDimerPotential)
    
    new_dist2_vector = zeros(length(config))
    return DimerPotentialVariable(new_dist2_vector,0. )
end

set_variables (generic function with 1 method)

In [ ]:
function dimer_energy_update(pos,index,config,dist2_mat,new_dist2_vec,new_en,en_tot,pot::AbstractDimerPotential)
    new_dist2_vec = [distance2(pos,b,config.bc) for b in config.pos]
    new_dist2_vec[index] = 0.
    delta_en = dimer_energy_atom(index,new_dist2_vec,pot) - dimer_energy_atom(index,dist2_mat, pot)

    return new_dist2_vec, delta_en + en_tot
end

function energy_update(pos,index,mc_state,pot::AbstractDimerPotential)

    mc_state.potential_variables.new_dist2_vec,mc_state.potential_variables.new_en = energy_update(trial_pos,index,mc_state.config,mc_state.dist2_mat,mc_state.potential_variables.new_dist2_vec,potential_variables.new_en,mc_state.en_tot,pot)

    return mc_state
end

# function get_energy_dimer(trial_pos,index,mc_state,pot::AbstractDimerPotential)
#     return energy_update(pos,index,mc_state,pot)
# end

function get_energy(trial_positions,indices,mc_states,pot::AbstractDimerPotential)
    return energy_update.(trial_positions,indices,mc_states,Ref(pot))
end

The NNPState struct becomes redundant, since we can add the various vectors to the potential variables

In [4]:
mutable struct NNPVariables <: PotentialVariables
    new_en::Float64
    new_dist2_vec::Vector
    new_en_atom::Vector
    g_matrix::Array
    f_matrix::Array
    new_g_matrix::Array
    new_f_vec::Vector
end
function set_variables(config,dist2_mat,pot::RuNNerPotential)
    
    n_atoms = length(config)
    f_matrix = cutoff_function.(sqrt.(dist2_mat),Ref(pot.r_cut))
    g_matrix = total_symm_calc(config.pos,dist2_mat,f_matrix,pot.symmetryfunctions)
    
    return NNPVariables(0. ,zeros(n_atoms),zeros(n_atoms),g_matrix,f_matrix,zeros(length(pot.symmetryfunctions)), zeros(n_atoms))
end

set_variables (generic function with 2 methods)

In [ ]:
function get_new_state_vars!(trial_pos,atomindex,nnp_state,pot)

    nnp_state.potential_variables.new_dist2_vec = [ distance2(trial_pos,b,nnp_state.config.bc) for b in nnp_state.config.pos]
    nnp_state.potential_variables.new_dist2_vec[atomindex] = 0.
    
    nnp_state.potential_variables.new_f_vec = cutoff_function.(sqrt.(nnp_state.potential_variables.new_dist2_vec),Ref(pot.r_cut))


    nnp_state.potential_variables.new_g_matrix = copy(nnp_state.potential_variables.g_matrix)

    nnp_state.potential_variables.new_g_matrix = total_thr_symm!(nnp_state.potential_variables.new_g_matrix,nnp_state.config.pos,trial_pos,nnp_state.dist2_mat,nnp_state.potential_variables.new_dist2_vec,nnp_state.potential_variables.f_matrix,nnp_state.potential_variables.new_f_vec,atomindex,pot.symmetryfunctions)

    return nnp_state
end
function calc_new_energy!(nnp_state,pot)
    nnp_state.potential_variables.new_en_atom = forward_pass(nnp_state.new_g_matrix,length(nnp_state.en_atom_vec),pot.nnp)
    return nnp_state
end
function get_runner_energy(trial_pos,index,state,pot::RuNNerPotential)
    state = get_new_state_vars(trial_pos,index,state,pot)
    state = calc_new_energy!(state,pot)
    state.potential_variables.new_en = sum(state.potential_variables.new_en_atom)
    return state
end
function get_energy(trial_positions,indices,mc_states,pot::RuNNerPotential)
    mc_states = get_runner_energy!.(trial_positions,indices,mc_states,Ref(pot))
    return mc_states
end

Similarly, the embedded atom model is defined as:

Some notes: This was the last potential written when I opted for thecurrent format. The functions are written with the form we're moving to in mind and only require a moderate amount of adjustment. 

In [5]:
mutable struct EmbeddedAtomVariables <: PotentialVariables
    new_en::Float64
    new_dist2_vec::Vector
    component_vector::Matrix
    new_component_vector::Matrix
end
function set_variables(config,dist2_matrix,pot::EmbeddedAtomPotential)
    
    return EmbeddedAtomVariables(zeros(length(config)),component_vector,new_component_vector)
end

set_variables (generic function with 3 methods)

In [ ]:
function calc_new_energy(mc_state,atomindex,newpos,pot::EmbeddedAtomPotential)

    mc_state.potential_variables.dist2_new = [distance2(newpos,b) for b in mc_state.config.pos]

    mc_state.potential_variables.dist2_new[atomindex] = 0.

    mc_state.potential_variables.new_component_vector = deepcopy(mc_state.potential_variables.component_vector)
    
    mc_state.potential_variables.new_component_vector = calc_components(mc_state.potential_variables.new_component_vector,atomindex,mc_state.dist2_mat[atomindex,:],mc_state.potential_variables.dist2_new,pot.n,pot.m)

    mc_state.potential_variables.new_en = calc_energies_from_components(new_component_vector,pot.ean,pot.eCam)

    return  mc_state
end

function get_energy(trial_positions,indices,mc_states,pot::EmbeddedAtomPotential)
    mc_states = calc_new_energy.(mc_states,indices,trial_positions,Ref(pot))
    return mc_states
end

And the Magnetic Field potential

In [6]:
mutable struct ELJPotentialB <: PotentialVariables
    new_en::Float64
    new_dist2_vec::Vector
    tanmat::Array
    new_tan_vec::Vector
end
function set_variables(config::Config,dist2_matrix::Matrix,pot::AbstractDimerPotentialB)
    n_atoms = length(config)
    tan_matrix = get_tantheta_mat(config,config.pos)

    return ELJPotentialB(zeros(n_atoms),tan_matrix,zeros(n_atoms))
end

set_variables (generic function with 4 methods)

we'll circle back to what happens with magnetic fields

Just to be cautious, writing MCState here and subbing it in


In [ ]:
mutable struct MCState{T,N,BC}
    temp::T
    beta::T
    config::Config{N,BC,T}
    dist2_mat::Matrix{T}

    potential_variables <:PotentialVariables
    # new_dist2_vector::Vector
    # new_energy::T

    en_tot::T
    ham::Vector{T}
    max_displ::Vector{T}
    max_boxlength::T
    count_atom::Vector{Int}
    count_vol::Vector{Int}
    count_rot::Vector{Int}
    count_exc::Vector{Int}
end
function MCState(
    temp, beta, config::Config{N,BC,T}, dist2_mat, potential_variables, en_tot; 
    max_displ = [0.1,0.1,1.], max_boxlength = 10., count_atom = [0,0], count_vol = [0,0], count_rot = [0,0], count_exc = [0,0]
) where {T,N,BC}
    ham = T[]
    MCState{T,N,BC}(
        temp, beta, deepcopy(config), copy(dist2_mat), copy(potential_variables), en_tot, ham, copy(max_displ), copy(max_boxlength), copy(count_atom), copy(count_vol), copy(count_rot), copy(count_exc)
        )
end
function MCState(temp, beta, config::Config, pot <:AbstractPotential ; kwargs...) 
    dist2_mat = get_distance2_mat(config)
    n_atoms = length(config.pos)

    # if typeof(config.bc) == PeriodicBC{Float64}
    #     en_atom_vec, en_tot = dimer_energy_config(dist2_mat, n_atoms, config.bc.box_length^2/4, pot)
    # else
    #     en_atom_vec, en_tot = dimer_energy_config(dist2_mat, n_atoms, pot)
    # end

# This section can't be predefined without actually generating the energy. In this way, we need to calculate the energy off the bat. Perhaps the best way to deal with this is have the get_energy function depend on the potential_variables rather than the 


    potential_variables = set_variables(config,dist2_mat,pot)
    MCState(temp, beta, config, dist2_mat, potential_variables, en_tot; kwargs...)
end